In [1]:
import os
import pandas as pd

main_publisher = 'Stanford'

script_dir = os.path.dirname(os.path.realpath('__file__'))
path = os.path.join(script_dir, '../Data/' + main_publisher + '/')
df_concepts = pd.read_csv(path + 'concepts_bis.csv', sep = '|')
print(df_concepts.shape)
df_concepts.head()

(16133, 3)


,OER,Concept,PR
0,0,http://en.wikipedia.org/wiki/Breast_cancer,0.006009
1,0,http://en.wikipedia.org/wiki/Gene,0.019077
2,0,http://en.wikipedia.org/wiki/Gene_expression,0.006689
3,0,http://en.wikipedia.org/wiki/Heat_map,0.005287
4,0,http://en.wikipedia.org/wiki/Machine_learning,0.007001


In [2]:
from SPARQLWrapper import SPARQLWrapper, JSON, XML
import time

def get_classes(concept):
    classes = []
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    sparql.setReturnFormat(JSON)
    request = """
                    SELECT ?class WHERE {
            <http://dbpedia.org/resource/CONCEPT> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> ?class
        }
    """
    request = request.replace('CONCEPT', concept)
    sparql.setQuery(request)
    done = False
    while not done : 
        try:
            ret = sparql.queryAndConvert()

            for r in ret["results"]["bindings"]:
                value = r["class"]["value"]
                if ("/yago/" not in value) and ("/umbel/" not in value) and (".wikidata." not in value):
                    classes.append({
                        "Concept" : concept,
                        "Class" : value
                    })
            done = True
            print(f'Concept {concept} done.')
        except Exception as e:
            print('Runtime Error : ', e)
            print('Sleeping for 10 seconds...')
            time.sleep(10)
            print('Sleep done !')
    return classes

In [3]:
y = [get_classes(x.split('/')[-1]) for x in df_concepts['Concept'].unique()]
y = [item for sublist in y for item in sublist]
df_classes = pd.DataFrame(y)
df_classes.head(20)
print(df_classes.shape)
df_classes.to_csv(path + "classes.csv", sep = '|')

Concept Breast_cancer done.
Concept Gene done.
Concept Gene_expression done.
Concept Heat_map done.
Concept Machine_learning done.
Concept Scatter_plot done.
Concept Spamming done.
Concept Academic_discipline done.
Concept Bayesian_inference done.
Concept Bit done.
Concept Cisco_(fish) done.
Concept Data_analysis done.
Concept Deep_learning done.
Concept Euclidean_vector done.
Concept False_discovery_rate done.
Concept Free_software done.
Concept Generalization done.
Concept Generalized_linear_model done.
Concept If_and_only_if done.
Concept Linear_regression done.
Concept Matrix_(mathematics) done.
Concept Matrix_completion done.
Concept Multiple_comparisons_problem done.
Concept Regression_analysis done.
Concept Statistics done.
Concept Survival_analysis done.
Concept Tree_structure done.
Concept Unsupervised_learning done.
Concept World_Wide_Web done.
Concept Concept done.
Concept Data done.
Concept Data_set done.
Concept Jonathan_Taylor_(American_football) done.
Concept Jupiter don